In [1]:
# ✅ Step 1: Upgrade pip, setuptools, and wheel first (important!)
%pip install --upgrade pip setuptools wheel

# ✅ Step 2: Install all required libraries (with fixed package names)
%pip install \
    "sentence-transformers>=2.2.2,<3.0" \
    langchain \
    flask \
    pypdf \
    python-dotenv \
    "pinecone-client[grpc]" \
    langchain-pinecone \
    langchain-community \
    langchain-openai \
    langchain-experimental


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pwd

'd:\\Introduction to Programming with Python\\PythonProjects\\Medical chat Bot\\research'

In [3]:
%pwd

'd:\\Introduction to Programming with Python\\PythonProjects\\Medical chat Bot\\research'

In [4]:
import os 
os.chdir("../")

In [5]:
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyMuPDFLoader
                             )
    document = loader.load()
    return document

In [7]:
%pwd

'd:\\Introduction to Programming with Python\\PythonProjects\\Medical chat Bot'

In [8]:
extracted_data = load_pdf(data='Data/')

In [9]:
# extracted_data

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("Length of Text chunks", len(text_chunks))

Length of Text chunks 5777


In [12]:
# text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
def download_hugging_face_embeddings():
    # Download the Hugging Face Embeddings
    emdeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return emdeddings

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\Muhammad Asim\AppData\Local\Temp\ipykernel_7772\2436550048.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emdeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
d:\Introduction to Programming with Python\PythonProjects\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
query_result = embeddings.embed_query("Hello World")
print("legth" , len(query_result))

legth 384


In [17]:
# query_result

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
import os

PINECONE_API_KEY = "pcsk_tcVhe_UNxCvkWUa1d5fTpKNjqp5aq7FtRC62y44Ek3Rs53wJxN2WeborV6PaSm8vVd44S"
PINECONE_ENVIRONMENT = "us-east-1"

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_ENVIRONMENT"] = PINECONE_ENVIRONMENT


In [20]:
# PINE_CONE_API_KEY = os.environ.get('PINE_CONE_API_KEY')

In [21]:
# PINE_CONE_API_KEY

In [22]:
# from pinecone.grpc import PineconeGRPC as pinecone
# from pinecone import ServerlessSpec

# pc = pinecone(api_key= PINE_CONE_API_KEY)
# index_name = "medicalchatbot"

# pc.create_index(
#     name= index_name,
#     dimension= 384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

In [23]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec

pc = pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalchatbot"

if index_name not in [i.name for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,  # this must match your embedding output dim
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


In [24]:
# import os
# os.environ["PINE_CONE_API_KEY"] = PINE_CONE_API_KEY

In [25]:
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name=index_name,
#     embedding=embeddings,
#     pinecone_api_key=PINE_CONE_API_KEY,
#     pinecone_environment="us-east-1"  # must match the one in ServerlessSpec
# )


In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)
